In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_core.prompts import PromptTemplate
from langchain.chains import create_sql_query_chain


In [2]:
load_dotenv()

True

In [3]:
# Setup
service_account_file = f"{os.getcwd()}/cns_dev_gcp.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_file

In [4]:
project='cdp-develop-347807'
dataset='pipst'
table_names_to_use=['l0lc_attribute_custom_key_affiliate','l0lc_behv_app_custom_key_affiliate','l0lc_behv_purchase_custom_key_affiliate']

In [5]:
# Connect to your Google BigQuery database
# db = SQLDatabase.from_uri("bigquery://cdp-develop-347807")
bigquery_url = f'bigquery://{project}/{dataset}?credentials_path={service_account_file}'
db = SQLDatabase.from_uri(bigquery_url)

In [6]:
# Create a language model
model = ChatOpenAI(model="gpt-4o", temperature=0)

In [7]:
chain = create_sql_query_chain(model, db,k=3)

In [8]:
generated_sql_query = chain.invoke({"question": "여성 고객 수를 구하세요",
                                    "table_names_to_use":table_names_to_use})
print(generated_sql_query.__repr__())

"```sql\nSELECT COUNT(*) AS female_customer_count\nFROM `l0lc_attribute_custom_key_affiliate`\nWHERE gender = 'F'\n```"


In [9]:
chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'dialect': 'bigquery', 'top_k': '3'}, template='Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a the few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also,